In [1]:
data_dir = './data/mlboot_dataset/'
results_dir = './results/'
import pandas as pd
train = pd.read_table(data_dir + 'mlboot_data.tsv', header=None, names=['uid','cat','f1','f2','f3','days'])

In [3]:
import scipy.sparse as sp
mat1 = sp.load_npz(data_dir+'mat1.npz')
mat2 = sp.load_npz(data_dir+'mat2.npz')
mat3 = sp.load_npz(data_dir+'mat3.npz')

In [11]:
import numpy as np
import gc
def key_func(x):
    ds1 = x.apply(lambda s: list(eval(s).keys())).sum()
    return len(np.unique(ds1))
df_uniq_cat_keys = train[['uid','cat','f1','f2','f3']].groupby(['uid','cat']).agg(key_func).reset_index()

In [46]:
df_uniq_cat_keys.head()

,uid,nuniq_keys_f1_cat0,nuniq_keys_f2_cat0,nuniq_keys_f3_cat0,nuniq_keys_f1_cat1,nuniq_keys_f2_cat1,nuniq_keys_f3_cat1,nuniq_keys_f1_cat2,nuniq_keys_f2_cat2,nuniq_keys_f3_cat2,...,nuniq_keys_f3_cat3,nuniq_keys_f1_cat4,nuniq_keys_f2_cat4,nuniq_keys_f3_cat4,nuniq_keys_f1_cat5,nuniq_keys_f2_cat5,nuniq_keys_f3_cat5,nuniq_keys_f1,nuniq_keys_f1,nuniq_keys_f1
0,00000d2994b6df9239901389031acaac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56.0,122.0,0.0,56,122,0
1,00000d53c06c43379ed22c775ce89614,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,21.0,0.0,7,21,0
2,00000e36ecd94e5c54ab9d08c7b29163,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,4.0,0.0,3,4,0
3,0000115cdd110ffa16d31dff81cf6272,0.0,0.0,0.0,41.0,67.0,0.0,9.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,84,0
4,0000115cdd110ffa16d31dff81cf6272,0.0,0.0,0.0,41.0,67.0,0.0,9.0,17.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,84,0


In [17]:
df_uniq_cat_keys.drop(['cat_y','nuniq_keys_f1_cat0','nuniq_keys_f2_cat0','nuniq_keys_f3_cat0'],inplace=True,axis=1)
df_uniq_cat_keys.columns = ['uid','cat','f1','f2','f3']
df_uniq_cat_keys.head()

,uid,cat,f1,f2,f3
0,00000d2994b6df9239901389031acaac,5,56,122,0
1,00000d53c06c43379ed22c775ce89614,5,7,21,0
2,00000e36ecd94e5c54ab9d08c7b29163,5,3,4,0
3,0000115cdd110ffa16d31dff81cf6272,1,41,67,0
4,0000115cdd110ffa16d31dff81cf6272,2,9,17,0


In [19]:
for c in range(6):
    q = df_uniq_cat_keys.loc[df_uniq_cat_keys['cat'] == c,['uid','cat','f1','f2','f3']].copy()
    q.columns = ['uid','cat','nuniq_keys_f1_cat'+str(c),'nuniq_keys_f2_cat'+str(c),'nuniq_keys_f3_cat'+str(c)]
    q.drop(['cat'],axis=1,inplace=True)
    df_uniq_cat_keys = df_uniq_cat_keys.merge(q, on='uid', how='left')
    del q
    df_uniq_cat_keys[['nuniq_keys_f1_cat'+str(c),'nuniq_keys_f2_cat'+str(c),'nuniq_keys_f3_cat'+str(c)]] =\
    df_uniq_cat_keys[['nuniq_keys_f1_cat'+str(c),'nuniq_keys_f2_cat'+str(c),'nuniq_keys_f3_cat'+str(c)]].fillna(0)

q = df_uniq_cat_keys[['uid','f1','f2','f3']].groupby('uid').agg('sum').reset_index()
q.columns = ['uid','nuniq_keys_f1','nuniq_keys_f1','nuniq_keys_f1']
df_uniq_cat_keys = df_uniq_cat_keys.merge(q, on='uid', how='left')
del q
df_uniq_cat_keys.head(n=10)

,uid,cat,f1,f2,f3,nuniq_keys_f1_cat0,nuniq_keys_f2_cat0,nuniq_keys_f3_cat0,nuniq_keys_f1_cat1,nuniq_keys_f2_cat1,...,nuniq_keys_f3_cat3,nuniq_keys_f1_cat4,nuniq_keys_f2_cat4,nuniq_keys_f3_cat4,nuniq_keys_f1_cat5,nuniq_keys_f2_cat5,nuniq_keys_f3_cat5,nuniq_keys_f1,nuniq_keys_f1,nuniq_keys_f1
0,00000d2994b6df9239901389031acaac,5,56,122,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56.0,122.0,0.0,56,122,0
1,00000d53c06c43379ed22c775ce89614,5,7,21,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,21.0,0.0,7,21,0
2,00000e36ecd94e5c54ab9d08c7b29163,5,3,4,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,4.0,0.0,3,4,0
3,0000115cdd110ffa16d31dff81cf6272,1,41,67,0,0.0,0.0,0.0,41.0,67.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,84,0
4,0000115cdd110ffa16d31dff81cf6272,2,9,17,0,0.0,0.0,0.0,41.0,67.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50,84,0
5,000014fe918d1f97a632a796f4948be8,2,77,503,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77,503,0
6,000039623dffd13f9cc93d27ffb2d152,2,60,430,2,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60,430,2
7,00006025b25e293ed4faa6c998f86180,5,44,181,4,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,44.0,181.0,4.0,44,181,4
8,0000ac9ba3f500d850a63a37cc5bea63,5,31,55,3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,31.0,55.0,3.0,31,55,3
9,0000b4d97dccff808111d8fa89cafdff,0,1,11,0,1.0,11.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,11,0


In [20]:
df_uniq_cat_keys.to_csv(data_dir + 'nuniq_keys.csv',index=False)

In [ ]:
del df_uniq_cat_keys

In [21]:
df.columns

Index(['uid', 'cat', 'num_keys_f1', 'sum_values_f1', 'num_keys_f2',
       'sum_values_f2', 'num_keys_f3', 'sum_values_f3', 'num_times_cat_eq_0',
       'num_times_cat_eq_1', 'num_times_cat_eq_2', 'num_times_cat_eq_3',
       'num_times_cat_eq_4', 'num_times_cat_eq_5', 'records', 'max_days',
       'min_days', 'sum_values_f1_std', 'num_keys_f1_std', 'sum_values_f2_std',
       'num_keys_f2_std', 'sum_values_f3_std', 'num_keys_f3_std',
       'sum_values_f1_max', 'num_keys_f1_max', 'sum_values_f2_max',
       'num_keys_f2_max', 'sum_values_f3_max', 'num_keys_f3_max',
       'sum_values_f1_mean', 'num_keys_f1_mean', 'sum_values_f2_mean',
       'num_keys_f2_mean', 'sum_values_f3_mean', 'num_keys_f3_mean'],
      dtype='object')

In [31]:
def sum_func(x):
    ds1 = x.apply(lambda s: np.sum(list(eval(s).values())))
    sum_values = ds1.sum()
    return sum_values

df_sum_cat = train[['uid','cat','f1','f2','f3']].groupby(['uid','cat']).agg(sum_func).reset_index()
for c in range(6):
    q = df_sum_cat.loc[df_sum_cat['cat'] == c,['uid','cat','f1','f2','f3']].copy()
    q.columns = ['uid','cat','sumval_keys_f1_cat'+str(c),'sumval_keys_f2_cat'+str(c),'sumval_keys_f3_cat'+str(c)]
    q.drop(['cat'],axis=1,inplace=True)
    df_sum_cat = df_sum_cat.merge(q, on='uid', how='left')
    del q
    df_sum_cat[['sumval_keys_f1_cat'+str(c),'sumval_keys_f2_cat'+str(c),'sumval_keys_f3_cat'+str(c)]] =\
    df_sum_cat[['sumval_keys_f1_cat'+str(c),'sumval_keys_f2_cat'+str(c),'sumval_keys_f3_cat'+str(c)]].fillna(0)

q = df_sum_cat[['uid','f1','f2','f3']].groupby('uid').agg('sum').reset_index()
q.columns = ['uid','sumval_keys_f1','sumval_keys_f1','sumval_keys_f1']
df_sum_cat = df_sum_cat.merge(q, on='uid', how='left')
del q
df_sum_cat.head(n=10)

,uid,cat,f1,f2,f3,sumval_keys_f1_cat0,sumval_keys_f2_cat0,sumval_keys_f3_cat0,sumval_keys_f1_cat1,sumval_keys_f2_cat1,...,sumval_keys_f3_cat3,sumval_keys_f1_cat4,sumval_keys_f2_cat4,sumval_keys_f3_cat4,sumval_keys_f1_cat5,sumval_keys_f2_cat5,sumval_keys_f3_cat5,sumval_keys_f1,sumval_keys_f1,sumval_keys_f1
0,00000d2994b6df9239901389031acaac,5,81.0,152.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,81.0,152.0,0.0,81.0,152.0,0.0
1,00000d53c06c43379ed22c775ce89614,5,11.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,23.0,0.0,11.0,23.0,0.0
2,00000e36ecd94e5c54ab9d08c7b29163,5,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0
3,0000115cdd110ffa16d31dff81cf6272,1,60.0,72.0,0.0,0.0,0.0,0.0,60.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,93.0,0.0
4,0000115cdd110ffa16d31dff81cf6272,2,12.0,21.0,0.0,0.0,0.0,0.0,60.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,93.0,0.0
5,000014fe918d1f97a632a796f4948be8,2,973.0,4136.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,973.0,4136.0,0.0
6,000039623dffd13f9cc93d27ffb2d152,2,315.0,1595.0,22.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.0,1595.0,22.0
7,00006025b25e293ed4faa6c998f86180,5,83.0,251.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,83.0,251.0,5.0,83.0,251.0,5.0
8,0000ac9ba3f500d850a63a37cc5bea63,5,48.0,60.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,48.0,60.0,3.0,48.0,60.0,3.0
9,0000b4d97dccff808111d8fa89cafdff,0,1.0,11.0,0.0,1.0,11.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11.0,0.0


In [33]:
qq = train[['uid','days','cat']].groupby(['uid','days']).agg('count').reset_index()
qq.columns = ['uid','days','max_day_cntr']
qq1 = qq[['uid','max_day_cntr']].groupby('uid').agg('max').reset_index()
qq1.columns = ['uid','max_day_cntr']
qq2 = qq[['uid','max_day_cntr']].groupby('uid').agg('mean').reset_index()
qq2.columns = ['uid','mean_day_cntr']
del qq

In [34]:
df = pd.read_csv(data_dir + 'preprocessed.csv') 
df = df[['uid', 'num_times_cat_eq_0',
       'num_times_cat_eq_1', 'num_times_cat_eq_2', 'num_times_cat_eq_3',
       'num_times_cat_eq_4', 'num_times_cat_eq_5', 'records', 'max_days',
       'min_days', 
       'sum_values_f1_max', 'num_keys_f1_max', 'sum_values_f2_max',
       'num_keys_f2_max', 'sum_values_f3_max', 'num_keys_f3_max',
       'sum_values_f1_mean', 'num_keys_f1_mean', 'sum_values_f2_mean',
       'num_keys_f2_mean', 'sum_values_f3_mean', 'num_keys_f3_mean']]
df = df.merge(qq1, on='uid', how='left')
df = df.merge(qq2, on='uid', how='left')
del qq1,qq2
df_uniq_cat_keys.drop(['cat','f1','f2','f3'],axis=1,inplace=True)
df = df.merge(df_uniq_cat_keys, on='uid', how='left')
df_sum_cat.drop(['cat','f1','f2','f3'],axis=1,inplace=True)
df = df.merge(df_sum_cat, on='uid', how='left')
#df.to_csv(data_dir + 'preprocessed_new.csv', index=False)
df.head(n=10)

,uid,num_times_cat_eq_0,num_times_cat_eq_1,num_times_cat_eq_2,num_times_cat_eq_3,num_times_cat_eq_4,num_times_cat_eq_5,records,max_days,min_days,...,sumval_keys_f3_cat3,sumval_keys_f1_cat4,sumval_keys_f2_cat4,sumval_keys_f3_cat4,sumval_keys_f1_cat5,sumval_keys_f2_cat5,sumval_keys_f3_cat5,sumval_keys_f1,sumval_keys_f1,sumval_keys_f1
0,00000d2994b6df9239901389031acaac,NaN,NaN,NaN,NaN,NaN,7.0,7,55,39,...,0.0,0.0,0.0,0.0,81.0,152.0,0.0,81.0,152.0,0.0
1,00000d53c06c43379ed22c775ce89614,NaN,NaN,NaN,NaN,NaN,2.0,2,15,14,...,0.0,0.0,0.0,0.0,11.0,23.0,0.0,11.0,23.0,0.0
2,00000e36ecd94e5c54ab9d08c7b29163,NaN,NaN,NaN,NaN,NaN,1.0,1,43,43,...,0.0,0.0,0.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0
3,0000115cdd110ffa16d31dff81cf6272,NaN,6.0,3.0,NaN,NaN,NaN,9,54,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,93.0,0.0
4,0000115cdd110ffa16d31dff81cf6272,NaN,6.0,3.0,NaN,NaN,NaN,9,54,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,93.0,0.0
5,0000115cdd110ffa16d31dff81cf6272,NaN,6.0,3.0,NaN,NaN,NaN,9,54,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,93.0,0.0
6,0000115cdd110ffa16d31dff81cf6272,NaN,6.0,3.0,NaN,NaN,NaN,9,54,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.0,93.0,0.0
7,000014fe918d1f97a632a796f4948be8,NaN,NaN,40.0,NaN,NaN,NaN,40,59,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,973.0,4136.0,0.0
8,000039623dffd13f9cc93d27ffb2d152,NaN,NaN,48.0,NaN,NaN,NaN,48,60,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,315.0,1595.0,22.0
9,00006025b25e293ed4faa6c998f86180,NaN,NaN,NaN,NaN,NaN,13.0,13,58,1,...,0.0,0.0,0.0,0.0,83.0,251.0,5.0,83.0,251.0,5.0


In [35]:
df = df.fillna(0)
df['most_freq_cat'] = np.argmax(df[[u'num_times_cat_eq_0', u'num_times_cat_eq_1',
       u'num_times_cat_eq_2', u'num_times_cat_eq_3', u'num_times_cat_eq_4',
       u'num_times_cat_eq_5']].fillna(0).values, axis=1)
df['diff_num_cats'] = (df['num_times_cat_eq_0']>0).astype(np.int32)+(df['num_times_cat_eq_1']>0).astype(np.int32)+\
(df['num_times_cat_eq_2']>0).astype(np.int32)+(df['num_times_cat_eq_3']>0).astype(np.int32)+\
(df['num_times_cat_eq_4']>0).astype(np.int32)+(df['num_times_cat_eq_5']>0).astype(np.int32)

In [41]:
def nun_func(x):
    return len(x.unique())
qq = train[['uid','days']].groupby('uid').agg(nun_func).reset_index()
qq.columns = ['uid','unique_days']

In [42]:
qq.head()

,uid,unique_days
0,00000d2994b6df9239901389031acaac,7
1,00000d53c06c43379ed22c775ce89614,2
2,00000e36ecd94e5c54ab9d08c7b29163,1
3,0000115cdd110ffa16d31dff81cf6272,8
4,000014fe918d1f97a632a796f4948be8,40


In [43]:
df = df.merge(qq, on='uid', how='left')
df.shape

(1057613, 69)

In [47]:
df = df.drop_duplicates()
df.to_csv(data_dir + 'preprocessed_new.csv', index=False)